# Cricket GNN Hyperparameter Search on Colab

This notebook runs hyperparameter optimization for the Cricket Ball Prediction GNN model using a free T4 GPU on Google Colab.

**Prerequisites:**
- Google account
- WandB account (free at wandb.ai)
- `data/t20s_male_json.zip` file (216 MB)

**Important:** Colab sessions have a 12-hour limit. Save your results frequently!

For detailed instructions, see `notes/training-guides/colab-guide.md`

## 1. GPU Check & Setup

First, verify that a GPU is available. You should see a T4 GPU listed.

In [ ]:
# Verify GPU is available
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n** WARNING: No GPU detected! **")
    print("Go to Runtime > Change runtime type > Hardware accelerator > T4 GPU")

## 2. Clone Repository

Clone the Cricket GNN repository from GitHub.

In [ ]:
!git clone https://github.com/lyndonkl/cricketmodel.git
%cd cricketmodel
!git log --oneline -3  # Show recent commits

## 3. Install Dependencies

Install PyTorch Geometric and other required packages. This may take a few minutes.

In [ ]:
# Install torch-geometric and dependencies
!pip install torch-geometric

# Install Optuna and WandB for hyperparameter search
!pip install optuna optuna-integration[wandb] wandb

# Install other dependencies
!pip install tqdm pyyaml scikit-learn plotly

print("\n" + "="*50)
print("Dependencies installed successfully!")
print("="*50)

## 4. WandB Login

Login to Weights & Biases for experiment tracking. You'll need your API key from [wandb.ai/authorize](https://wandb.ai/authorize).

In [ ]:
import wandb

# This will prompt you to enter your API key
# Get it from: https://wandb.ai/authorize
wandb.login()

## 5. Upload & Extract Raw Data

Upload the `t20s_male_json.zip` file (216 MB). When you run this cell, a file upload dialog will appear.

**Note:** The upload may take a few minutes depending on your internet connection.

In [ ]:
from google.colab import files
import os

# Upload t20s_male_json.zip when prompted
print("Please upload 't20s_male_json.zip' (216 MB)...")
uploaded = files.upload()

# Check what was uploaded
for filename in uploaded.keys():
    print(f"Uploaded: {filename} ({len(uploaded[filename]) / 1e6:.1f} MB)")

# Extract to data directory
!mkdir -p data
!unzip -q t20s_male_json.zip -d data/

# Verify extraction
json_count = len([f for f in os.listdir('data/t20s_male_json') if f.endswith('.json')])
print(f"\nExtracted {json_count} JSON match files to data/t20s_male_json/")

## 6. Process Data (One-Time, ~30 min)

This cell processes the raw JSON data into PyTorch graphs. **This only needs to run once per session** - the processed data is cached.

**Expected time:** ~30 minutes on first run

In [ ]:
import time

print("Processing data - this takes ~30 minutes on first run...")
print("The data will be cached for the rest of this session.")
print("="*50)

start_time = time.time()

# Run minimal HP search to trigger data processing
!python scripts/hp_search.py --phase phase1_coarse --n-trials 1 --epochs 1 --device cuda

elapsed = time.time() - start_time
print("\n" + "="*50)
print(f"Data processing complete! ({elapsed/60:.1f} minutes)")
print("="*50)

## 7. Run HP Search

Now run the actual hyperparameter search. Adjust `--n-trials` and `--epochs` based on your time budget.

**Recommended settings:**
- Quick test: `--n-trials 5 --epochs 10` (~30 min)
- Standard: `--n-trials 10 --epochs 25` (~2-3 hours)
- Thorough: `--n-trials 20 --epochs 30` (~5-6 hours)

In [ ]:
# Phase 1: Coarse search (GPU-accelerated)
# Adjust --n-trials and --epochs based on your time budget

!python scripts/hp_search.py \
    --phase phase1_coarse \
    --n-trials 10 \
    --epochs 25 \
    --wandb \
    --device cuda \
    --n-jobs 1

## 8. View Results

Display the best hyperparameters found during the search.

In [ ]:
import json
import glob

# Find the most recent best_params.json
param_files = glob.glob('checkpoints/optuna/cricket_gnn_phase1_coarse_*/best_params.json')

if param_files:
    # Get most recent file
    latest_file = max(param_files, key=os.path.getmtime)
    print(f"Results from: {latest_file}")
    print("="*50)
    
    with open(latest_file) as f:
        best_params = json.load(f)
    
    for key, value in best_params.items():
        print(f"{key}: {value}")
else:
    print("No results found yet. Run the HP search first.")

## 9. Download Results

Download all checkpoints and results to your local machine for further analysis or to continue training later.

In [ ]:
from google.colab import files
import os

# Create results archive
!zip -r results.zip checkpoints/optuna/ optuna_studies.db

# Show archive contents
!unzip -l results.zip | head -20
print("...")

# Get file size
size_mb = os.path.getsize('results.zip') / 1e6
print(f"\nTotal archive size: {size_mb:.1f} MB")

# Download
print("\nStarting download...")
files.download('results.zip')

## Optional: Continue with Phase 2

If you have time remaining in your Colab session, continue to Phase 2 (architecture tuning).

In [ ]:
import glob

# Find Phase 1 results
phase1_files = glob.glob('checkpoints/optuna/cricket_gnn_phase1_coarse_*/best_params.json')

if phase1_files:
    latest_phase1 = max(phase1_files, key=os.path.getmtime)
    print(f"Using Phase 1 results from: {latest_phase1}")
    
    # Run Phase 2
    !python scripts/hp_search.py \
        --phase phase2_architecture \
        --n-trials 12 \
        --epochs 25 \
        --best-params "{latest_phase1}" \
        --wandb \
        --device cuda \
        --n-jobs 1
else:
    print("No Phase 1 results found. Run Phase 1 first.")

## Troubleshooting

**GPU not detected:**
- Go to Runtime > Change runtime type > Hardware accelerator > T4 GPU

**Out of memory:**
- Reduce batch size: add `--batch-size 32` to the hp_search command
- Restart runtime: Runtime > Restart runtime

**Session disconnected:**
- Re-run cells 1-5 (GPU check through data upload)
- Cell 6 (data processing) will use cached data if available
- Your Optuna study is saved to SQLite and can be resumed

**WandB errors:**
- Run `wandb.login()` again if your session expired